# Tutorial 1: RDME- Bimolecular Reaction
## 1-1 Uniformly Distribution

## 0.Environment Check
if you didn't install the environments in docker or any container, we suggest you do it through anaconda/miniconda.

basically, you need to install the dependencies
```bash
conda env create -f rdme_env.yml
```

Then import jLM package.


In [ ]:
import jLM                                  # Set up the Jupyter environment
from jLM.RDME import Sim as RDMESim         # Main simulation class
from jLM.RegionBuilder import RegionBuilder # Deal with the spatial geometry
from lm import IntMpdRdmeSolver             # lm::rdme::IntMpdRdmeSolver

import numpy as np

# 1. Initialization
## 1.1 RDME simulation object creation

First step is to create a object that contains all the essential information to start a simulation.

The class we use here is:

``` python
sim = jLM.RDME.Sim(simulation_name, 
                    filename, 
                    dimensions, 
                    latticeSpacing, 
                    regionName, 
                    dt=None)
````
+ simulation_name is what we call our system;
+ filename: the name of the final output trajectory file.(hdf5 format)
+ dimensions: the number of lattices in x,y,z dimension. in list [dimx,dimy,dimz], must be divisible by 32.
+ latticeSpacing: the actual physical representation of the length of each subvolume cube, unit: meter.
+ the Name of the entire sim region, default="external"
+ dt: time steps for the RDME simualtion, unit: second.

In [ ]:
cellvol = 1.0e-15                # L=dm^3
simname = "Bimolecular_System_uni"
filename = "T1-1_bimolecular_uni.lm"
Ldim = [64,64,64]                   # Has to be multiples of 32
lattice_spacing = 32e-9            # in m
regionN = "extracellular"
dt = 25e-6                          # in s

In [ ]:
sim = RDMESim(  simname, 
                filename, 
                Ldim, 
                lattice_spacing, 
                regionN,
                dt)

in case you forget, or want to redefine some key parameters in the simulation we define above, you can always call methods to adjust them.

for example, we can change the simulation time step by:

In [ ]:
sim.timestep = 50e-6            # in s

next we need to define the total simulation time, the time interval to write results to our trajectory for lattice and species, normally we keep them the same.

Be careful, the unit for Interval is "time steps", and should be an integer.

The actual physical time for interval would be: 
$$
t = Interval * dt
$$
since we define $dt=50 \times 10^{-6} s$, the actual write interval would be:

$$
20000 * 50 \times 10^{-6} = 0.1s
$$


In [ ]:
sim.simulationTime=30.0     # unit is seconds

sim.latticeWriteInterval= int(2000)    # unit is timesteps, dt
sim.speciesWriteInterval= int(2000)    # unit is timesteps, dt

## 1.2 Spatial Geometry Initialization

now, we need to add more spatial compartments to our system, for simplicity, our first toy system would only have three spatial regions:
+ extracellular
+ cytoplasm
+ plasm membrane



we want each of them mutually excluded, and if we combine them all together, it should be the entire lattice sites we create:

$$\mathtt{lattice} = \mathtt{extracellular} \cup \mathtt{membrane}\cup \mathtt{cytoplasm} $$
and naturally
$$\emptyset= \mathtt{membrane} \cap \mathtt{cytoplasm}$$
$$\emptyset= \mathtt{membrane} \cap \mathtt{extracellular}$$
$$\emptyset= \mathtt{cytoplasm} \cap \mathtt{extracellular}$$


In order to design the site lattice geometry, we create a `RegionBuilder` object, which reads the lattice dimensions 
from our simulation object `sim`.

In [ ]:
build = RegionBuilder(sim)

we create a sphere to represent the cytoplasm with radius=25, center at [32,32,32]

In [ ]:
radius = int(np.ceil((cellvol*3/4/np.pi)**(1/3)*0.1/lattice_spacing))
print(radius)
cytoplasm = build.ellipsoid(radius = radius, center = [32,32,32])

Then we dilate the cytoplasm, and create a surface with the dilation method, and exclude the cytoplasm to represent our membrane

In [ ]:
cytoplasm_dilation = build.dilate(cytoplasm, se = build.se26)
membrane = cytoplasm_dilation & ~cytoplasm

Finally, make sure our extracellular region excludes the cell.

In [ ]:
# effectively, cytoplasm_dilation = cytoplasm and  membrane
extracellular = ~cytoplasm_dilation

we need our simulation system to compose all the regions together.

In [ ]:
cyt = sim.region('cytoplasm')
mem = sim.region('membrane')
ext = sim.region('extracellular')

In [ ]:
build.compose(
    (sim.region('extracellular'), extracellular),
    (sim.region('cytoplasm'), cytoplasm),
    (sim.region('membrane'), membrane))

wonderful, now we have all the geometric infomation, we can visualize it in two different ways: stacks and 3D

In [ ]:
sim.showRegionStack()

In [ ]:
sim.displayGeometry()

## 1.2 Species and diffusion Co. Initialization

For simplicity, we only want our bimolecular reaction take place in cytoplasm and only allow each species to diffuse in cytoplasm.

Our Template will be a reversible reaction as: 

$$ A + B \rightleftharpoons C $$

However, our software only accept one-way reaction, we need to separate it into two irreversible reactions: 

$$ A + B \xrightarrow{k_f} C $$
$$ C \xrightarrow{k_r} A + B$$

initial condition: 

#A = 1000

#B = 1000

#C = 0


In [ ]:
spA = sim.species('A')
spB = sim.species('B')
spC = sim.species('C')


+ function references:

`sim.species((str)name, [(str,latex)textPepr, (str)annotation])`



Then you can call the following function to check the relevant info about your species:

In [ ]:
sim.showAllSpecies()

In [ ]:
sim.showSpecies('A')

Now, you may notice all Diffusion rates are undefined, so we are going to **define all the diffusion coefficients** for each species.

In [ ]:
sim.transitionRate(None, None, None, sim.diffusionZero) # initialization of the diffusion rates to 0

define a custom made diffusion coefficient.

In [ ]:
sim.diffusionConst('diff1', 1e-14)

set diffusion rate:

In [ ]:
sim.transitionRate(sim.species('A'), sim.region('cytoplasm'), sim.region('cytoplasm'), sim.dc.diff1)
# sim.transitionRate(spA, mem, mem, sim.dc.diff1)

sim.transitionRate(spB, cyt, cyt, sim.dc.diff1)    # use defined diffusion rate
sim.transitionRate(spC, cyt, cyt, sim.dc.diff1)


check the result:

In [ ]:
sim.showAllSpecies()

## 1.3 Reactions Initialization 

In [ ]:


NA = 6.022e23      # molecules/mole
# kf = sim.rateConst('kf', 1.07e5/(NA*cellvol), 2)    
kf = sim.rateConst('kf', 1.07e5, 2)  
# rate constant for the reaction A + B -> C, second order, /M/s
kr = sim.rateConst('kr', 0.351 , 1)   
# rate constant for the reaction C -> A + B, first order, /s

Reactions can be added by the function: `jLM.RDME.Sim.region('region_name').addReaction`

Since we already define the name of the region we want to add reactions, there will be two ways you can add reactions:

In [ ]:
sim.region('cytoplasm').addReaction([spA,spB], [spC], sim.rc.kf)
cyt.addReaction([spC], [spA,spB], kr)

Distribute a concentration of particles uniformly through cytoplasm:

In [ ]:
sim.distributeNumber(spA, cyt, 1000)
sim.distributeNumber(spB, cyt, 1000)
sim.distributeNumber(spC, cyt, 0)
# sim.sp.A.placeNumberInto(cyt, 1000)

In [ ]:
sim.showAllSpecies()

# 2 Check the system state

In [ ]:
sim

now we confirmed it is what we want, we can now proceed to finalize the simulation we create.

In [ ]:
sim.finalize()

# 3. Run the Simulation

In [ ]:
sim.run(solver=IntMpdRdmeSolver(), cudaDevices=[0])

# 4. Analysis

The trajectory will be saved as h5 format, we need the python package `h5py` to access it, or you can download the [`HDFView`](https://www.hdfgroup.org/downloads/) to see it in a GUI.

In [ ]:
import matplotlib.pyplot as plt
import h5py
import seaborn as sns
from jLM.RDME import File as RDMEFile

In [ ]:
# traj_file = h5py.File('T1_bimolecular.lm ', 'r')
traj = RDMEFile(filename)
ts, As = traj.getNumberTrajectory(species="A")
ts, Bs = traj.getNumberTrajectory(species="B")
ts, Cs = traj.getNumberTrajectory(species="C")

In [ ]:
# now visuzlize the trajectories with seaborn and plt
sns.set(style="whitegrid")  
# Create a color palette
palette = sns.color_palette()  
plt.figure(figsize=(10,6))
plt.plot(ts, As, label='A',marker='o', linestyle='-',color=palette[0])
plt.plot(ts, Bs, label='B',marker='x', linestyle='--',color=palette[1])
plt.plot(ts, Cs, label='C',color=palette[2])

plt.title('Trajectories of Biomolecular System uniform distribution')
plt.xlabel('Time (s)')
plt.ylabel('Counts')
plt.legend()
plt.savefig('T1-1_bimolecular_uni.png')
plt.show()